## Programming for Data Analysis Project Notebook 2020 - Killian Foley
### Problem Statement
Manufacturing engineer performing a simulation on drilling of a hole in a workpiece to be tapped for an M8 x 1.25 thread screw in a future process. Rework is not acceptable so 1 iteration of drilling only. 
From reference "https://www.engineeringtoolbox.com/metric-threads-d_777.html" - we find tap size is 6.8mm with tolerance grade of IT7/H7 (https://www.engineersedge.com/international_tol.htm) so minimum hole diamter is 6.8mm and maximum hole diamter is 6.815 mm anything outside of.
Drill bit tolerance from manufacturers specification sheet - https://www.maford.com/SiteContent/Documents//2018_catalog_sections//MAFord_2018_Master_Catalog_HP_Drill_%20Selection_Chart.pdf - IT7/h7  6.8 +0/ -0.015mm. 

Drilling - Cutting process where the drill 

#### Variables Explained
Is the hole within tolerance = "pass" (True or False)  
Drill bit starting diameter in mm = "drill_diameter" (mm)  
Number of cycles of drill bit = "cycles" (unit less number)  
Hole diameter = "hole_diameter" = (mm)

In [1]:
#start by importing libraries
import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns
%matplotlib inline

# set Numpy precision to 3 decimal places due to practical measurement accuracies
np.set_printoptions(precision=3)

#setting the random generator so results can be reproduced

from numpy.random import default_rng
rg = default_rng(135792468)

#Generating drill diameter data
drill_diameter = rg.uniform(6.8,6.815,100)
drill_diameter

array([6.809, 6.813, 6.805, 6.803, 6.81 , 6.808, 6.809, 6.803, 6.813,
       6.814, 6.808, 6.805, 6.803, 6.814, 6.8  , 6.809, 6.803, 6.813,
       6.808, 6.814, 6.811, 6.802, 6.815, 6.806, 6.805, 6.815, 6.802,
       6.812, 6.805, 6.804, 6.8  , 6.801, 6.813, 6.812, 6.812, 6.812,
       6.804, 6.814, 6.814, 6.808, 6.808, 6.809, 6.807, 6.811, 6.808,
       6.813, 6.809, 6.803, 6.803, 6.813, 6.813, 6.804, 6.808, 6.809,
       6.806, 6.806, 6.8  , 6.814, 6.813, 6.813, 6.811, 6.812, 6.811,
       6.801, 6.815, 6.805, 6.807, 6.803, 6.815, 6.807, 6.808, 6.802,
       6.809, 6.809, 6.814, 6.811, 6.812, 6.801, 6.811, 6.806, 6.813,
       6.804, 6.801, 6.812, 6.807, 6.805, 6.804, 6.803, 6.811, 6.803,
       6.801, 6.811, 6.803, 6.813, 6.805, 6.804, 6.813, 6.808, 6.806,
       6.811])